<a href="https://colab.research.google.com/github/chiyunlee/Predicting-Prices-of-Taiwanese-Commercial-Real-Estate/blob/main/model_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as T
import torchvision.models
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import numpy as np
import pandas as pd
from PIL import Image
from sklearn import preprocessing, metrics, model_selection

In [3]:
USE_GPU = True

dtype = torch.float # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


In [4]:
train_mean = 0.0
train_std = 0.0
val_mean = 0.0
val_std = 0.0

In [5]:
# load images
from os import listdir, path

images_list = listdir('/content/drive/MyDrive/SC201_Final_Project/images/map')

print(images_list)

image_dic_index = {}
image_dic_latlng = {}
for i in range(len(images_list)):
  image_dic_index[i] = images_list[i][:len(images_list[i])-4]
  image_dic_latlng[images_list[i][:len(images_list[i])-4]] = i
print(image_dic_index)
print(image_dic_latlng)

['25.06881_121.58975.png', '25.06039_121.52206.png', '25.06753_121.52709.png', '25.04685_121.50418.png', '25.08212_121.59265.png', '25.03415_121.52653.png', '25.13663_121.54661.png', '25.08224_121.55351.png', '25.04463_121.5269.png', '25.0587_121.56444.png', '25.04415_121.56649.png', '25.04524_121.54365.png', '25.0878_121.52637.png', '25.06301_121.528.png', '25.05606_121.52681.png', '25.04953_121.52608.png', '25.11857_121.53437.png', '25.03838_121.49893.png', '24.99011_121.5427.png', '25.04601_121.50897.png', '25.03138_121.51986.png', '25.04606_121.50874.png', '25.0463_121.51582.png', '25.0658_121.52328.png', '25.04698_121.52981.png', '25.04902_121.52802.png', '25.04426_121.50341.png', '24.9914_121.54384.png', '25.04629_121.53271.png', '25.11859_121.53841.png', '25.13794_121.50298.png', '25.024_121.52914.png', '24.98887_121.56038.png', '25.06251_121.51681.png', '25.03374_121.50517.png', '25.06729_121.5974.png', '25.02815_121.50194.png', '25.08282_121.5532.png', '25.12125_121.4674.png',

In [6]:
nan_cache = {}
def data_preprocess(filename,mode='Train'):
  data = pd.read_csv(filename)
  data.insert(len(data.columns), 'index', '')
  data.loc[data['lat_lng'] == '25_121.50227','lat_lng'] = '25.0_121.50227'
  index_list = []
  for index, row in data.iterrows():
    index_list.append(image_dic_latlng[row[2]])
  data['index'] = index_list
  if mode == 'Train':
    # split data to train, validation and test datsets
    train_data, val_data = model_selection.train_test_split(data,test_size = 0.2)
    means = train_data.mean() # to use mean or other indicators to fillna?
    train_data = train_data.fillna(means)
    val_data = val_data.fillna(means)
    for column in data:
      nan_cache[column] = means
    train_y = train_data.pop('unit_price')
    val_y = val_data.pop('unit_price')
    return train_data, val_data, train_y, val_y
  elif mode == 'Test':
    real_test_data = data.fillna(nan_cache)
    return real_test_data

In [7]:
district_list = ['villages_towns_Sanxia', 'villages_towns_Sanzhi',
       'villages_towns_Sanchong', 'villages_towns_Zhonghe',
       'villages_towns_Zhongshan', 'villages_towns_Zhongzheng',
       'villages_towns_Wugu', 'villages_towns_Xinyi', 'villages_towns_Neihu',
       'villages_towns_Bali', 'villages_towns_Beitou',
       'villages_towns_Nangang', 'villages_towns_Tucheng',
       'villages_towns_Shilin', 'villages_towns_Datong', 'villages_towns_Daan',
       'villages_towns_Wenshan', 'villages_towns_Xindian',
       'villages_towns_Xinzhuang', 'villages_towns_Songshan',
       'villages_towns_Banqiao', 'villages_towns_Linkou',
       'villages_towns_Shulin', 'villages_towns_Yonghe',
       'villages_towns_Xizhi', 'villages_towns_Taishan',
       'villages_towns_Tamsui', 'villages_towns_Shenkeng',
       'villages_towns_Ruifang', 'villages_towns_Wanhua',
       'villages_towns_Wanli', 'villages_towns_Luzhou',
       'villages_towns_Gongliao', 'villages_towns_Jinshan',
       'villages_towns_Shuangxi', 'villages_towns_Yingge']
taipei_district_list = ['villages_towns_Nangang','villages_towns_Beitou','villages_towns_Shilin', 'villages_towns_Datong', 'villages_towns_Daan',
       'villages_towns_Wenshan','villages_towns_Songshan', 'villages_towns_Wanhua','villages_towns_Zhongshan', 'villages_towns_Zhongzheng','villages_towns_Xinyi', 'villages_towns_Neihu']
xinbei_district_list = ['villages_towns_Sanxia', 'villages_towns_Sanzhi',
       'villages_towns_Sanchong', 'villages_towns_Zhonghe',
       'villages_towns_Zhongshan', 'villages_towns_Zhongzheng',
       'villages_towns_Wugu','villages_towns_Bali', 'villages_towns_Tucheng','villages_towns_Xindian','villages_towns_Xinzhuang','villages_towns_Banqiao', 'villages_towns_Linkou',
       'villages_towns_Shulin', 'villages_towns_Yonghe',
       'villages_towns_Xizhi', 'villages_towns_Taishan',
       'villages_towns_Tamsui', 'villages_towns_Shenkeng',
       'villages_towns_Ruifang','villages_towns_Wanli', 'villages_towns_Luzhou',
       'villages_towns_Gongliao', 'villages_towns_Jinshan',
       'villages_towns_Shuangxi', 'villages_towns_Yingge']
building_material = ['building_materials_RC', 'building_materials_RB',
       'building_materials_brick', 'building_materials_steel',
       'building_materials_SRC', 'building_materials_PRX',
       'building_materials_other_material']

In [8]:
FILE = '/content/drive/MyDrive/SC201_Final_Project/Data/final_data_0830.csv'

In [9]:
train_data, val_data, train_y, val_y = data_preprocess(FILE,mode='Train')
print(val_data.shape)


(5536, 91)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  del sys.path[0]


In [10]:
train_data.head()

,lat,lng,lat_lng,zoning,total_floors,floors_area,room,hall,bathroom,compartmented,...,building_state_other_state,building_materials_SC,building_materials_RC,building_materials_RB,building_materials_brick,building_materials_steel,building_materials_SRC,building_materials_PRX,building_materials_other_material,index
21071,25.051706,121.581205,25.0517055_121.5812047,-0.054178,-0.705334,-0.054441,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2094
21831,25.054130,121.485440,25.05413_121.48544,-0.285405,-0.304740,-0.235401,2,2,1,1,...,0,0,1,0,0,0,0,0,0,2279
22040,25.197050,121.429720,25.19705_121.42972,-0.195064,1.698233,0.080844,3,2,2,1,...,0,0,1,0,0,0,0,0,0,1136
20145,25.055900,121.523800,25.0559_121.5238,-0.305304,-0.304740,-0.394533,2,1,1,1,...,0,0,1,0,0,0,0,0,0,1757
6765,24.946360,121.381700,24.94636_121.3817,0.198540,0.362918,0.392962,4,2,2,1,...,0,0,1,0,0,0,0,0,0,2779


In [11]:
val_data.head()

,lat,lng,lat_lng,zoning,total_floors,floors_area,room,hall,bathroom,compartmented,...,building_state_other_state,building_materials_SC,building_materials_RC,building_materials_RB,building_materials_brick,building_materials_steel,building_materials_SRC,building_materials_PRX,building_materials_other_material,index
21788,25.00727,121.45610,25.00727_121.4561,-0.412361,-0.304740,-0.350646,1,1,1,1,...,0,0,1,0,0,0,0,0,0,813
11617,25.00569,121.51697,25.00569_121.51697,0.079942,-0.705334,0.003938,5,0,5,1,...,0,0,1,0,0,0,0,0,0,360
14165,25.04836,121.54443,25.04836_121.54443,-0.306896,-0.171208,-0.211279,2,2,1,1,...,0,0,1,0,0,0,0,0,0,1668
19154,25.05819,121.52432,25.05819_121.52432,-0.437434,-0.571803,-0.431541,0,0,0,0,...,0,0,1,0,0,0,0,0,0,131
7586,25.19454,121.42928,25.19454_121.42928,-0.243617,1.030575,-0.026100,2,2,1,1,...,0,0,1,0,0,0,0,0,0,1102


In [12]:
# standardize data
train_mean = 154170.694
train_std = 79570.40139
to_drop = ['zoning', 'lat','lng','lat_lng','unit_berth_price','compartmented','management_committee','floors_area','establishment','clothing_store','home_goods_store','store','local_government_office','university','natural_feature','health','tourist_attraction','transit_station','food']
for drop_items in to_drop:
  train_data.pop(drop_items)
  val_data.pop(drop_items)
  
print(train_data.columns)
print(val_data.shape)

Index(['total_floors', 'room', 'hall', 'bathroom', 'total_berth_price',
       'main_building_area', 'auxiliary_building_area', 'balcony_area',
       'elevator', 'building_age', 'travel_agency', 'point_of_interest',
       'transaction_first_floor', 'shifting_level', 'villages_towns_Sanxia',
       'villages_towns_Sanzhi', 'villages_towns_Sanchong',
       'villages_towns_Zhonghe', 'villages_towns_Zhongshan',
       'villages_towns_Zhongzheng', 'villages_towns_Wugu',
       'villages_towns_Xinyi', 'villages_towns_Neihu', 'villages_towns_Bali',
       'villages_towns_Beitou', 'villages_towns_Nangang',
       'villages_towns_Tucheng', 'villages_towns_Shilin',
       'villages_towns_Datong', 'villages_towns_Daan',
       'villages_towns_Wenshan', 'villages_towns_Xindian',
       'villages_towns_Xinzhuang', 'villages_towns_Songshan',
       'villages_towns_Banqiao', 'villages_towns_Linkou',
       'villages_towns_Shulin', 'villages_towns_Yonghe',
       'villages_towns_Xizhi', 'villages_t

In [13]:
val_data.head()

,total_floors,room,hall,bathroom,total_berth_price,main_building_area,auxiliary_building_area,balcony_area,elevator,building_age,...,building_state_other_state,building_materials_SC,building_materials_RC,building_materials_RB,building_materials_brick,building_materials_steel,building_materials_SRC,building_materials_PRX,building_materials_other_material,index
21788,-0.304740,1,1,1,-0.301478,-0.301478,-0.301478,-0.301478,1,-0.301478,...,0,0,1,0,0,0,0,0,0,813
11617,-0.705334,5,0,5,-0.335347,-0.335347,-0.335347,-0.335347,1,-0.335347,...,0,0,1,0,0,0,0,0,0,360
14165,-0.171208,2,2,1,0.469211,0.469211,0.469211,0.469211,1,0.469211,...,0,0,1,0,0,0,0,0,0,1668
19154,-0.571803,0,0,0,-0.240902,-0.240902,-0.240902,-0.240902,1,-0.240902,...,0,0,1,0,0,0,0,0,0,131
7586,1.030575,2,2,1,-1.019395,-1.019395,-1.019395,-1.019395,1,-1.019395,...,0,0,1,0,0,0,0,0,0,1102


In [14]:
# load images to dictionary
def load_imgs(lat_lng):
  image = Image.open('/content/drive/MyDrive/SC201_Final_Project/images/map/{}.png'.format(lat_lng)).convert('RGB')
  image = T.ToTensor()(image)
  return image

In [15]:
class HousingDataset(torch.utils.data.Dataset):
  def __init__(self,dataset,labels):
    self.y_train = torch.tensor(labels.values,dtype=torch.float32).view(len(labels),1)
    self.x_train = torch.tensor(dataset.values,dtype=torch.float32)
    print("HousingDataset shape {}, {}".format(self.x_train.shape, self.y_train.shape))
  def __len__(self):
    return len(self.y_train)
  def __getitem__(self,idx):
    index = int(self.x_train[idx][71])
    # print(index)      # how to extract values from tensor
    x_img = load_imgs(image_dic_index[index])
    return self.x_train[idx], x_img , self.y_train[idx]

In [16]:
# Datasets load training examples one at a time, so we wrap each Dataset in a 
# DataLoader which iterates through the Dataset and forms minibatches. We divide
# the training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
# This is for after map data is loaded into the features, so shuffling is done
BATCH_SIZE = 32
def form_minibatch(data,mode='Train'):
  if mode == 'Train':
    data = DataLoader(data,batch_size=BATCH_SIZE,shuffle=True)
  elif mode == 'Test':
    data = DataLoader(data,batch_size=BATCH_SIZE)
  return data

train_dl = form_minibatch(HousingDataset(train_data,train_y))
eval_dl = form_minibatch(HousingDataset(val_data,val_y),mode='Test')
print("training size {}, eval size {}".format(len(train_dl), len(eval_dl)))

HousingDataset shape torch.Size([22140, 72]), torch.Size([22140, 1])
HousingDataset shape torch.Size([5536, 72]), torch.Size([5536, 1])
training size 692, eval size 173


In [17]:
# model = MyModel()
class MyModel(nn.Module):
    def __init__(self,input_size):
        super().__init__()
        self.nn = nn.Sequential(
        nn.Linear(input_size, 60),
        nn.BatchNorm1d(60),
        nn.ReLU(),
        nn.Linear(60, 30),
        nn.BatchNorm1d(30),
        nn.ReLU(),
        nn.Linear(30, 15),
        nn.BatchNorm1d(15),
        nn.ReLU(),
        nn.Linear(15, 8),
        nn.BatchNorm1d(8),
        )
        # epoch 25-29
        resnet = torchvision.models.resnet50(pretrained=True)
        for param in resnet.parameters():
          param.requires_grad = False
        

        self.features2 = nn.Sequential(
              nn.AdaptiveAvgPool2d((224,224)),
              *list(resnet.children())[:6],

              # # N x 2048 * 28 * 28
              # nn.Conv2d(in_channels=2048, out_channels=1024, kernel_size=3, padding=1),
              # nn.BatchNorm2d(1024),
              # nn.ReLU(),

              # # N x 1024 * 28 * 28
              # nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=3, padding=1),
              # nn.BatchNorm2d(512),
              # nn.ReLU(),

              # N x 512 * 28 * 28
              nn.Conv2d(in_channels=512, out_channels=256, kernel_size=3, padding=1),
              nn.BatchNorm2d(256),
              nn.ReLU(),

              # N x 256 * 28 * 28
              nn.Conv2d(in_channels=256, out_channels=128, kernel_size=3, padding=1),
              nn.BatchNorm2d(128),
              nn.ReLU(),

              # N x 128 * 28 * 28
              nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, padding=1),
              nn.BatchNorm2d(64),
              nn.ReLU(),

              # N x 64 x 28 x 28  
              nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, padding=1),
              nn.BatchNorm2d(32),
              nn.ReLU(),

              nn.MaxPool2d(kernel_size=2, stride=2),  

              # N x 32 x 14 x 14
              nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3, padding=1),
              nn.BatchNorm2d(16),
              nn.ReLU(),

              # N x 16 x 14 x 14
              nn.Conv2d(in_channels=16, out_channels=8, kernel_size=3, padding=1),
              nn.BatchNorm2d(8),
              nn.ReLU(),

              nn.MaxPool2d(kernel_size=2, stride=2),

              # N x 8 x 7 x 7
              
              nn.Flatten(),
              # nn.Linear(2048,8)
              # nn.Linear(2048, 256),
              # nn.BatchNorm1d(256),
              # nn.ReLU(),
              nn.Linear(8*7*7,16),
              # nn.BatchNorm1d(16),



          )
        self.metrics = 0
        self.fc_out = nn.Sequential(
            nn.Linear(24,16),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.BatchNorm1d(8),
            nn.ReLU(),
            nn.Linear(8, 4),
            nn.BatchNorm1d(4),
            nn.ReLU(),
            nn.Linear(4,1)
        )


    def forward(self,x1,x2):
        x1 = self.nn(x1)
        x1 = x1.view(x1.size(0), -1)
        x1 = F.relu(x1)

        x2 = self.features2(x2)
        x2 = x2.view(x2.size(0), -1)
        x2 = F.relu(x2)
        # Concatenate in dim1 (feature dimension)
        x = torch.cat((x1, x2),1)
        scores = self.fc_out(x)
        return scores

net = MyModel(71)
optimizer = torch.optim.Adam(net.parameters(),lr=5e-4,weight_decay=0.99)
# lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10)
optimizer_SGD_momentum = torch.optim.SGD(net.parameters(),lr=1e-5,momentum=0.9,weight_decay=0.98)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [18]:
loss_history=[]
val_loss=[]
def train(data1, data2, model, optimizer, epochs=1):
    """
    Train a model on real estate data property features 
    and extracte map features using the PyTorch Module API.
    
    Inputs:
    - data1: training data
    - data2: validation data
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    criterion = nn.L1Loss()
    for e in range(epochs):
        running_loss = 0.0
        for t, (x,x2,y) in enumerate(data1):
        # for t, (x1,y) in enumerate(data1):
            model.train()  # put model to training mode
            indices = torch.tensor(range(0, 71))    # how to drop column from tensor
            x1 = torch.index_select(x, 1, indices)
            x1 = x1.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            # print(x1.size())
            x2 = x2.to(device=device,dtype=dtype)
            y = y.to(device=device, dtype=dtype)
            
            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()
            
            scores = model(x1,x2)
            # scores = model(x1)
            # print(scores)
            loss = criterion(scores,y)
            running_loss += loss.item()
            loss_history.append(loss)
            # running_loss += loss.item()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()
            training_loss = running_loss/len(data1)
            if t % 100 == 0:
                running_loss = 0.0
                print('Iteration %d, loss = %.4f' % (t, training_loss))
                print('Checking accuracy on validation set')
                eval_loss, eval_l1_loss = check_accuracy(data2, model)
                print("Epoch {}, Eval loss: {}, l1 loss: {}".format(e, eval_loss, eval_l1_loss))
        # lr_scheduler.step(model.metrics)


In [19]:
def check_accuracy(data, model):
    criterion = nn.L1Loss() 
    total_loss = 0.0
    total_l1_loss = 0.0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
      for x,x2,y in data:
      # for x1,y in data:
        indices = torch.tensor(range(0, 71))
        x1 = torch.index_select(x, 1, indices)
        # print(x1.size())
        x1 = x1.to(device=device, dtype=dtype)  # move to device, e.g. GPU
        x2 = x2.to(device=device,dtype=dtype)
        y = y.to(device=device, dtype=dtype)
        scores = model(x1,x2)
        # scores = model(x1)
        loss = criterion(scores,y)
        total_loss += loss.item()
        ground_truth_price = y * train_std + train_mean
        predicted_price = scores * train_std + train_mean
        total_l1_loss += abs(ground_truth_price - predicted_price)
      val_loss.append(total_l1_loss.data[0][0]/len(data))
      return total_loss/len(data), total_l1_loss.data[0][0]/len(data)

In [ ]:
train(train_dl, eval_dl, net,optimizer,epochs=5)

Iteration 0, loss = 0.0012
Checking accuracy on validation set
Epoch 0, Eval loss: 0.7266568066067778, l1 loss: 53137.0859375


In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(loss_history)),loss_history)
plt.xlabel('iteration')
plt.ylabel('training loss')
plt.title('Training Loss history')
plt.show()

In [ ]:
plt.plot(range(len(val_loss)),val_loss)
plt.xlabel('iteration')
plt.ylabel('val_l1_loss')
plt.title('Validation Loss History')
plt.show()

In [ ]:
# def predict(data,model):
#     model.eval()  # set model to evaluation mode
#     data = data_preprocess(data,mode='Test')
#     data = standardize(data)
#     data = data.to_numpy()
#     data = torch.from_numpy(data)
#     loader_data = form_minibatch(data,mode='Test')
#     with torch.no_grad():
#       for x1,x2 in loader_data:
#         x1 = x1.to(device=device, dtype=dtype)  # move to device, e.g. GPU
#         x2 = x2.to(device=device, dtype=dtype)
#         scores = model(x1,x2)
#     return scores

In [ ]:
# sort image according to train_data
lat_data = train_data['lat']
lng_data = train_data['lng']
for i in range(len(lat_data)):
  if i == 0:
    temp_lat = lat_data.iloc[i]
    temp_lng = lng_data.iloc[i]
    if f'{temp_lat}_{temp_lng}' in image_dic:
      sorted_imgs = image_dic[f'{temp_lat}_{temp_lng}']
    elif f'{round(temp_lat,3)}_{round(temp_lng,3)}' in image_dic:
      sorted_imgs = image_dic[f'{round(temp_lat,3)}_{round(temp_lng,3)}']
    elif f'{round(temp_lat,4)}_{round(temp_lng,4)}' in image_dic:
      sorted_imgs = image_dic[f'{round(temp_lat,4)}_{round(temp_lng,4)}']
    elif f'{round(temp_lat,5)}_{round(temp_lng,5)}' in image_dic:
      sorted_imgs = image_dic[f'{round(temp_lat,5)}_{round(temp_lng,5)}']
    elif f'{round(temp_lat,6)}_{round(temp_lng,6)}' in image_dic:
      sorted_imgs = image_dic[f'{round(temp_lat,6)}_{round(temp_lng,6)}']
    elif f'{round(temp_lat,7)}_{round(temp_lng,7)}' in image_dic:
      sorted_imgs = image_dic[f'{round(temp_lat,7)}_{round(temp_lng,7)}']
    print(sorted_imgs.size())
  else:
    temp_lat = lat_data.iloc[i]
    temp_lng = lng_data.iloc[i]
    if f'{temp_lat}_{temp_lng}' in image_dic:
      sorted_imgs = torch.stack((sorted_imgs,image_dic[f'{temp_lat}_{temp_lng}']),0)
    elif f'{round(temp_lat,3)}_{round(temp_lng,3)}' in image_dic:
      sorted_imgs = torch.stack((sorted_imgs,image_dic[f'{round(temp_lat,3)}_{round(temp_lng,3)}']),0)
    elif f'{round(temp_lat,4)}_{round(temp_lng,4)}' in image_dic:
      sorted_imgs = torch.stack((sorted_imgs,image_dic[f'{round(temp_lat,4)}_{round(temp_lng,4)}']),0)
    elif f'{round(temp_lat,5)}_{round(temp_lng,5)}' in image_dic:
      sorted_imgs = torch.stack((sorted_imgs,image_dic[f'{round(temp_lat,5)}_{round(temp_lng,5)}']),0)
    elif f'{round(temp_lat,6)}_{round(temp_lng,6)}' in image_dic:
      sorted_imgs = torch.stack((sorted_imgs,image_dic[f'{round(temp_lat,6)}_{round(temp_lng,6)}']),0)
    elif f'{round(temp_lat,7)}_{round(temp_lng,7)}' in image_dic:
      sorted_imgs = torch.stack((sorted_imgs,image_dic[f'{round(temp_lat,7)}_{round(temp_lng,7)}']),0)
    print(sorted_imgs.size())